# CSV Processor Pro: Data Analysis Dashboard
This notebook provides an interactive interface to analyze and visualize your CSV data.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scripts.kpi_analyzer import calculate_kpis
from scripts.visualizer import generate_visualizations

# Load data
df = pd.read_csv('data/large_sample.csv')
print(f"Loaded {len(df)} rows of data.")
df.head()

## Key Performance Indicators (KPIs)

In [ ]:
kpis = calculate_kpis(df)
print("--- Summary KPIs ---")
for k, v in kpis.items():
    print(f"{k.replace('_', ' ').title()}: {v:,.2f}" if isinstance(v, float) else f"{k.replace('_', ' ').title()}: {v}")

## Visualizations

In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.histplot(df['salary'], kde=True, color='green')
plt.title('Overall Salary Distribution')
plt.show()